In [1]:
import torch.nn as nn
import torch
import pytorch_lightning as pl
from utils import  get_sobel_kernel, get_gaussian_kernel, load_letter_conv_weights
from models import CustomTransposedConv2d

class LetterFilter(pl.LightningModule):
    def __init__(self,config):
        super().__init__()
        self.device = config["device"]
        letter_conv_k = config["letter_conv_k"]
        letter_conv_stride = config["letter_conv_stride"]
        sobel_k = config["sobel_k"]
        gauss_sig = config["gauss_sig"]
        font_path = config["font_path"]
        letters = config["letters"]
        gauss_k = letter_conv_k

        self.letter_size_weight = config["letter_size_weight"]
        self.detail_weight = config["detail_weight"]

        # sobel edge filter
        sobel_2D = get_sobel_kernel(sobel_k)
        self.sobel_filter_x = nn.Conv2d(in_channels=1,
                                        out_channels=1,
                                        kernel_size=sobel_k,
                                        padding=sobel_k // 2,
                                        bias=False)
        self.sobel_filter_x.weight[:] = torch.from_numpy(sobel_2D)


        self.sobel_filter_y = nn.Conv2d(in_channels=1,
                                        out_channels=1,
                                        kernel_size=sobel_k,
                                        padding=sobel_k // 2,
                                        bias=False)
        self.sobel_filter_y.weight[:] = torch.from_numpy(sobel_2D.T)

        #gaussian fitler after sobel
        gaussian_2D = get_gaussian_kernel(gauss_k, 0, gauss_sig)
        self.gaussian_filter = nn.Conv2d(in_channels=1,
                                         out_channels=1,
                                         kernel_size=gauss_k,
                                         stride=letter_conv_stride,
                                         padding=gauss_k // 2,
                                         bias=False)
        self.gaussian_filter.weight[:] = torch.from_numpy(gaussian_2D)

        # transposed convs for mask
        transposed_convs_weights = load_letter_conv_weights(font_path, letter_conv_k, letters)
        transposed_padding = 
        self.transp_conv = CustomTransposedConv2d(transposed_convs_weights, transposed_convs_weights.shape[0], 1, letter_conv_k, letter_conv_stride, transposed_padding)
        






/home/peter/repositories/Typewriter/Image2Letter/.venv/lib64/python3.11/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/home/peter/repositories/Typewriter/Image2Letter/.venv/lib64/python3.11/site-packages/pydantic/_internal/_config.py:269: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
